In [1]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=8a6fb7ddf74a40e1ae773e9b9f72c8899823161f46ff4a393637d8a44546f5c0
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [2]:
from __future__ import print_function
from collections import OrderedDict

import torch
import torch.nn as nn
from torch.nn import init
from torch.autograd import Variable
from torch import autograd

import time
import _pickle as cPickle

import urllib
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 80
plt.style.use('seaborn-pastel')

import os
import sys
import codecs
import re
import numpy as np

#import các thư viện cần thiết

import numpy as np
import pandas as pd
import torch

#Embedding
from transformers import BertTokenizerFast, BertForTokenClassification, BertModel

#Modeling
from torch.utils.data import DataLoader
from torch.optim import SGD, Adam
from seqeval.metrics import classification_report
from tqdm import tqdm

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/tmp/ipykernel_42/244750557.py:16: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-pastel')
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/kaggle/input/covid19vi/word/dev_word.conll
/kaggle/input/covid19vi/word/train_word.conll
/kaggle/input/covid19vi/word/test_word.json
/kaggle/input/covid19vi/word/dev_word.json
/kaggle/input/covid19vi/word/test_word.conll
/kaggle/input/covid19vi/word/train_word.json
/kaggle/input/covid19vi/syllable/test_syllable.json
/kaggle/input/covid19vi/syllable/dev_syllable.json
/kaggle/input/covid19vi/syllable/test_syllable.conll
/kaggle/input/covid19vi/syllable/train_syllable.conll
/kaggle/input/covid19vi/syllable/dev_syllable.conll
/kaggle/input/covid19vi/syllable/train_syllable.json
/kaggle/input/word2vec-vi-words-100dims/word2vec_vi_words_100dims.txt
/kaggle/input/word2vec-vi-syllables-100dims/word2vec_vi_syllables_100dims.txt


In [3]:
#parameters for the Model
parameters = OrderedDict()
parameters['train'] = "/kaggle/input/covid19vi/syllable/train_syllable.conll" #Path to train file
parameters['dev'] = "/kaggle/input/covid19vi/syllable/dev_syllable.conll" #Path to test file
parameters['test'] = "/kaggle/input/covid19vi/syllable/test_syllable.conll" #Path to dev file
#parameters['tag_scheme'] = "BIOES" #BIO or BIOES
parameters['lower'] = True # Boolean variable to control lowercasing of words
parameters['zeros'] =  True # Boolean variable to control replacement of  all digits by 0 
parameters['char_dim'] = 50 #Char embedding dimension
parameters['word_dim'] = 100 #Token embedding dimension
parameters['word_lstm_dim'] = 200 #Token LSTM hidden layer size
parameters['word_bidirect'] = True #Use a bidirectional LSTM for words
parameters['embedding_path'] = "/kaggle/input/word2vec-vi-syllables-100dims/word2vec_vi_syllables_100dims.txt" #Location of pretrained embeddings
parameters['all_emb'] = 1 #Load all embeddings
parameters['crf'] =1 #Use CRF (0 to disable)
parameters['dropout'] = 0.25 #Droupout on the input (0 = no dropout)
parameters['epoch'] =  30 #Number of epochs to run"
parameters['weights'] = "" #path to Pretrained for from a previous run
parameters['name'] = "BCC_syllable" # Model name
#parameters['gradient_clip']=5.0
parameters['char_mode']="CNN"
models_path = "./models/" #path to saved models

#GPU
parameters['use_gpu'] = torch.cuda.is_available() #GPU Check
use_gpu = parameters['use_gpu']

#parameters['reload'] = "/kaggle/input/named-entity-recognition/pre-trained-model" 

#Constants
START_TAG = '<START>'
STOP_TAG = '<STOP>'

In [4]:
#paths to files 
# #To stored mapping file
# mapping_file = '/kaggle/input/named-entity-recognition/mapping.pkl'

#To stored model
name = parameters['name']
model_name = models_path + name #get_name(parameters)

if not os.path.exists(models_path):
    os.makedirs(models_path)

In [5]:
#hàm đọc dữ liệu
def read_dataset(file_path):
    tokens=[]
    ner_tags=[]
    ids=[]
    count=1
    with open(file_path) as f:
        lines=f.readlines()
        ts=[]
        nts=[]
        for line in lines:
            line = line.split()
            if len(line)==0:
                ids.append(count)
                tokens.append(ts)
                ner_tags.append(nts)
                ts=[]
                nts=[]
                count+=1
            else:
                ts.append(line[0])
                nts.append(line[-1])
    data = pd.DataFrame({'Id':ids, 'NER_tags':ner_tags, 'Tokens':tokens})
    return data

In [6]:
def get_NER_labels(data):
    NERs = list(data['NER_tags'].values)
    labels_list = []
    for value in NERs:
        labels_list = labels_list + value
    types = list(set(labels_list))
    return types

In [7]:
train_df = read_dataset('/kaggle/input/covid19vi/syllable/train_syllable.conll')
val_df = read_dataset('/kaggle/input/covid19vi/syllable/dev_syllable.conll')
test_df = read_dataset('/kaggle/input/covid19vi/syllable/test_syllable.conll')

In [8]:
words_list = train_df['Tokens'].values.tolist()+val_df['Tokens'].values.tolist()+test_df['Tokens'].values.tolist()
words = set([t for l in words_list for t in l])

In [9]:
chars = set([c for word in words for c in word])

In [10]:
unique_labels = get_NER_labels(train_df)

In [11]:
unique_labels

['O',
 'B-ORGANIZATION',
 'B-DATE',
 'B-JOB',
 'I-TRANSPORTATION',
 'B-SYMPTOM_AND_DISEASE',
 'B-LOCATION',
 'I-DATE',
 'B-GENDER',
 'I-SYMPTOM_AND_DISEASE',
 'I-AGE',
 'B-TRANSPORTATION',
 'I-LOCATION',
 'I-PATIENT_ID',
 'B-NAME',
 'I-GENDER',
 'I-ORGANIZATION',
 'B-PATIENT_ID',
 'I-JOB',
 'B-AGE',
 'I-NAME']

In [12]:
def create_dico(words):
    """
    Create a dictionary of items from a list of list of items.
    """
    dico = {}
    for word in words:
        if word not in dico:
            dico[word] = 1
        else:
            dico[word] += 1
    return dico

def create_mapping(dico):
    """
    Create a mapping (item to ID / ID to item) from a dictionary.
    Items are ordered by decreasing frequency.
    """
    sorted_items = sorted(dico.items(), key=lambda x: (-x[1], x[0]))
    id_to_item = {i: v[0] for i, v in enumerate(sorted_items)}
    item_to_id = {v: k for k, v in id_to_item.items()}
    return item_to_id, id_to_item

def word_mapping(words):
    """
    Create a dictionary and a mapping of words, sorted by frequency.
    """
    dico = create_dico(words)
    dico['<UNK>'] = 10000000 #UNK tag for unknown words
    word_to_id, id_to_word = create_mapping(dico)
    print("Found %i unique words" % len(dico))
    return dico, word_to_id, id_to_word

def char_mapping(chars):
    """
    Create a dictionary and mapping of characters, sorted by frequency.
    """
    dico = create_dico(chars)
    char_to_id, id_to_char = create_mapping(dico)
    print("Found %i unique characters" % len(dico))
    return dico, char_to_id, id_to_char

def tag_mapping(unique_labels):
    """
    Create a dictionary and a mapping of tags, sorted by frequency.
    """
    dico = create_dico(unique_labels)
    dico[START_TAG] = -1
    dico[STOP_TAG] = -2
    tag_to_id, id_to_tag = create_mapping(dico)
    print("Found %i unique named entity tags" % len(dico))
    return dico, tag_to_id, id_to_tag

In [13]:
dico_words,word_to_id,id_to_word = word_mapping(words)
dico_chars, char_to_id, id_to_char = char_mapping(chars)
dico_tags, tag_to_id, id_to_tag = tag_mapping(unique_labels)

Found 5728 unique words
Found 180 unique characters
Found 23 unique named entity tags


# Chuẩn bị dữ liệu

In [14]:
def prepare_dataset(sentences, labels, word_to_id, char_to_id, tag_to_id):
    """
    Prepare the dataset. Return a list of lists of dictionaries containing:
        - word indexes
        - word char indexes
        - tag indexes
    """
    data = []
    for s, label in zip(sentences,labels):
        str_words = s
        words = [word_to_id[w if w in word_to_id else '<UNK>'] for w in str_words]
        chars = [[char_to_id[c] for c in w if c in char_to_id]
                 for w in str_words]
        tags = [tag_to_id[l] for l in label]
        data.append({
            'str_words': str_words,
            'words': words,
            'chars': chars,
            'tags': tags,
        })
    return data

train_data = prepare_dataset(
    train_df['Tokens'].values.tolist(), train_df['NER_tags'].values.tolist(), word_to_id, char_to_id, tag_to_id
)
dev_data = prepare_dataset(
    val_df['Tokens'].values.tolist(), val_df['NER_tags'].values.tolist(), word_to_id, char_to_id, tag_to_id
)
test_data = prepare_dataset(
    test_df['Tokens'].values.tolist(), test_df['NER_tags'].values.tolist(), word_to_id, char_to_id, tag_to_id
)
print("{} / {} / {} sentences in train / dev / test.".format(len(train_data), len(dev_data), len(test_data)))

5027 / 2000 / 3000 sentences in train / dev / test.


In [15]:
train_data[0]

{'str_words': ['Đồng',
  'thời',
  ',',
  'bệnh',
  'viện',
  'tiếp',
  'tục',
  'thực',
  'hiện',
  'các',
  'biện',
  'pháp',
  'phòng',
  'chống',
  'dịch',
  'bệnh',
  'COVID',
  '-',
  '19',
  'theo',
  'hướng',
  'dẫn',
  'của',
  'Bộ',
  'Y',
  'tế',
  '.'],
 'words': [5538,
  5014,
  11,
  3607,
  5283,
  5038,
  5244,
  5022,
  4044,
  3780,
  3532,
  4684,
  4693,
  3735,
  3915,
  3607,
  1818,
  12,
  392,
  4927,
  4098,
  3908,
  3845,
  1795,
  3495,
  5221,
  13],
 'chars': [[103, 148, 64, 57],
  [70, 58, 155, 59],
  [8],
  [52, 142, 64, 58],
  [72, 59, 142, 64],
  [70, 59, 136, 66],
  [70, 160, 53],
  [70, 58, 167, 53],
  [58, 59, 142, 64],
  [53, 86, 53],
  [52, 59, 142, 64],
  [66, 58, 86, 66],
  [66, 58, 94, 64, 57],
  [53, 58, 147, 64, 57],
  [54, 144, 53, 58],
  [52, 142, 64, 58],
  [27, 39, 46, 33, 28],
  [9],
  [13, 21],
  [70, 58, 55, 65],
  [58, 110, 154, 64, 57],
  [54, 126, 64],
  [53, 161, 51],
  [26, 152],
  [49],
  [70, 136],
  [10]],
 'tags': [20,
  20,


# Load word embeddings

In [16]:
all_word_embeds = {}
for i, line in enumerate(codecs.open(parameters['embedding_path'], 'r', 'utf-8')):
    s = line.strip().split()
    if len(s) == parameters['word_dim'] + 1:
        all_word_embeds[s[0]] = np.array([float(i) for i in s[1:]])

In [17]:
#Intializing Word Embedding Matrix
word_embeds = np.random.uniform(-np.sqrt(0.06), np.sqrt(0.06), (len(word_to_id), parameters['word_dim']))

for w in word_to_id:
    if w in all_word_embeds:
        word_embeds[word_to_id[w]] = all_word_embeds[w]
    elif w.lower() in all_word_embeds:
        word_embeds[word_to_id[w]] = all_word_embeds[w.lower()]

print('Loaded %i pretrained embeddings.' % len(all_word_embeds))

Loaded 977025 pretrained embeddings.


# Mô hình hóa

**Viết hàm khởi tạo embeddings cho char level**

In [18]:
def init_embedding(input_embedding):
    """
    Initialize embedding
    """
    bias = np.sqrt(3.0 / input_embedding.size(1))
    nn.init.uniform(input_embedding, -bias, bias)

**Viết hàm khởi tạo trọng số cho ma trận trong lớp linear layer khi dự đoán đầu ra**

In [19]:
def init_linear(input_linear):
    """
    Initialize linear transformation
    """
    bias = np.sqrt(6.0 / (input_linear.weight.size(0) + input_linear.weight.size(1)))
    nn.init.uniform(input_linear.weight, -bias, bias)
    if input_linear.bias is not None:
        input_linear.bias.data.zero_()

**Viết hàm khởi tạo trọng số trong các lớp LSTM**

In [20]:
def init_lstm(input_lstm):
    """
    Initialize lstm
    
    PyTorch weights parameters:
    
        weight_ih_l[k]: the learnable input-hidden weights of the k-th layer,
            of shape `(hidden_size * input_size)` for `k = 0`. Otherwise, the shape is
            `(hidden_size * hidden_size)`
            
        weight_hh_l[k]: the learnable hidden-hidden weights of the k-th layer,
            of shape `(hidden_size * hidden_size)`            
    """
    
    # Weights init for forward layer
    for ind in range(0, input_lstm.num_layers):
        
        ## Gets the weights Tensor from our model, for the input-hidden weights in our current layer
        weight = eval('input_lstm.weight_ih_l' + str(ind))
        
        # Initialize the sampling range
        sampling_range = np.sqrt(6.0 / (weight.size(0) / 4 + weight.size(1)))
        
        # Randomly sample from our samping range using uniform distribution and apply it to our current layer
        nn.init.uniform(weight, -sampling_range, sampling_range)
        
        # Similar to above but for the hidden-hidden weights of the current layer
        weight = eval('input_lstm.weight_hh_l' + str(ind))
        sampling_range = np.sqrt(6.0 / (weight.size(0) / 4 + weight.size(1)))
        nn.init.uniform(weight, -sampling_range, sampling_range)
        
        
    # We do the above again, for the backward layer if we are using a bi-directional LSTM (our final model uses this)
    if input_lstm.bidirectional:
        for ind in range(0, input_lstm.num_layers):
            weight = eval('input_lstm.weight_ih_l' + str(ind) + '_reverse')
            sampling_range = np.sqrt(6.0 / (weight.size(0) / 4 + weight.size(1)))
            nn.init.uniform(weight, -sampling_range, sampling_range)
            weight = eval('input_lstm.weight_hh_l' + str(ind) + '_reverse')
            sampling_range = np.sqrt(6.0 / (weight.size(0) / 4 + weight.size(1)))
            nn.init.uniform(weight, -sampling_range, sampling_range)

    # Bias initialization steps
    
    # We initialize them to zero except for the forget gate bias, which is initialized to 1
    if input_lstm.bias:
        for ind in range(0, input_lstm.num_layers):
            bias = eval('input_lstm.bias_ih_l' + str(ind))
            
            # Initializing to zero
            bias.data.zero_()
            
            # This is the range of indices for our forget gates for each LSTM cell
            bias.data[input_lstm.hidden_size: 2 * input_lstm.hidden_size] = 1
            
            #Similar for the hidden-hidden layer
            bias = eval('input_lstm.bias_hh_l' + str(ind))
            bias.data.zero_()
            bias.data[input_lstm.hidden_size: 2 * input_lstm.hidden_size] = 1
            
        # Similar to above, we do for backward layer if we are using a bi-directional LSTM 
        if input_lstm.bidirectional:
            for ind in range(0, input_lstm.num_layers):
                bias = eval('input_lstm.bias_ih_l' + str(ind) + '_reverse')
                bias.data.zero_()
                bias.data[input_lstm.hidden_size: 2 * input_lstm.hidden_size] = 1
                bias = eval('input_lstm.bias_hh_l' + str(ind) + '_reverse')
                bias.data.zero_()
                bias.data[input_lstm.hidden_size: 2 * input_lstm.hidden_size] = 1

**Những hàm cho việc sử dụng CRF**

In [21]:
def log_sum_exp(vec):
    '''
    This function calculates the score explained above for the forward algorithm
    vec 2D: 1 * tagset_size
    '''
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))
    
def argmax(vec):
    '''
    This function returns the max index in a vector
    '''
    _, idx = torch.max(vec, 1)
    return to_scalar(idx)

def to_scalar(var):
    '''
    Function to convert pytorch tensor to a scalar
    '''
    return var.view(-1).data.tolist()[0]

In [22]:
def score_sentences(self, feats, tags):
    # tags is ground_truth, a list of ints, length is len(sentence)
    # feats is a 2D tensor, len(sentence) * tagset_size
    r = torch.LongTensor(range(feats.size()[0]))
    if self.use_gpu:
        r = r.cuda()
        pad_start_tags = torch.cat([torch.cuda.LongTensor([self.tag_to_ix[START_TAG]]), tags])
        pad_stop_tags = torch.cat([tags, torch.cuda.LongTensor([self.tag_to_ix[STOP_TAG]])])
    else:
        pad_start_tags = torch.cat([torch.LongTensor([self.tag_to_ix[START_TAG]]), tags])
        pad_stop_tags = torch.cat([tags, torch.LongTensor([self.tag_to_ix[STOP_TAG]])])

    score = torch.sum(self.transitions[pad_stop_tags, pad_start_tags]) + torch.sum(feats[r, tags])

    return score

In [23]:
def forward_alg(self, feats):
    '''
    This function performs the forward algorithm
    '''
    # calculate in log domain
    # feats is len(sentence) * tagset_size
    # initialize alpha with a Tensor with values all equal to -10000.
    
    # Do the forward algorithm to compute the partition function
    init_alphas = torch.Tensor(1, self.tagset_size).fill_(-10000.)
    
    # START_TAG has all of the score.
    init_alphas[0][self.tag_to_ix[START_TAG]] = 0.
    
    # Wrap in a variable so that we will get automatic backprop
    forward_var = autograd.Variable(init_alphas)
    if self.use_gpu:
        forward_var = forward_var.cuda()
        
    # Iterate through the sentence
    for feat in feats:
        # broadcast the emission score: it is the same regardless of
        # the previous tag
        emit_score = feat.view(-1, 1)
        
        # the ith entry of trans_score is the score of transitioning to
        # next_tag from i
        tag_var = forward_var + self.transitions + emit_score
        
        # The ith entry of next_tag_var is the value for the
        # edge (i -> next_tag) before we do log-sum-exp
        max_tag_var, _ = torch.max(tag_var, dim=1)
        
        # The forward variable for this tag is log-sum-exp of all the
        # scores.
        tag_var = tag_var - max_tag_var.view(-1, 1)
        
        # Compute log sum exp in a numerically stable way for the forward algorithm
        forward_var = max_tag_var + torch.log(torch.sum(torch.exp(tag_var), dim=1)).view(1, -1) # ).view(1, -1)
    terminal_var = (forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]).view(1, -1)
    alpha = log_sum_exp(terminal_var)
    # Z(x)
    return alpha

In [24]:
def viterbi_algo(self, feats):
    '''
    In this function, we implement the viterbi algorithm.
    A Dynamic programming based approach to find the best tag sequence
    '''
    backpointers = []
    # analogous to forward
    
    # Initialize the viterbi variables in log space
    init_vvars = torch.Tensor(1, self.tagset_size).fill_(-10000.)
    init_vvars[0][self.tag_to_ix[START_TAG]] = 0
    
    # forward_var at step i holds the viterbi variables for step i-1
    forward_var = Variable(init_vvars)
    if self.use_gpu:
        forward_var = forward_var.cuda()
    for feat in feats:
        next_tag_var = forward_var.view(1, -1).expand(self.tagset_size, self.tagset_size) + self.transitions
        _, bptrs_t = torch.max(next_tag_var, dim=1)
        bptrs_t = bptrs_t.squeeze().data.cpu().numpy() # holds the backpointers for this step
        next_tag_var = next_tag_var.data.cpu().numpy() 
        viterbivars_t = next_tag_var[range(len(bptrs_t)), bptrs_t] # holds the viterbi variables for this step
        viterbivars_t = Variable(torch.FloatTensor(viterbivars_t))
        if self.use_gpu:
            viterbivars_t = viterbivars_t.cuda()
            
        # Now add in the emission scores, and assign forward_var to the set
        # of viterbi variables we just computed
        forward_var = viterbivars_t + feat
        backpointers.append(bptrs_t)

    # Transition to STOP_TAG
    terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
    terminal_var.data[self.tag_to_ix[STOP_TAG]] = -10000.
    terminal_var.data[self.tag_to_ix[START_TAG]] = -10000.
    best_tag_id = argmax(terminal_var.unsqueeze(0))
    path_score = terminal_var[best_tag_id]
    
    # Follow the back pointers to decode the best path.
    best_path = [best_tag_id]
    for bptrs_t in reversed(backpointers):
        best_tag_id = bptrs_t[best_tag_id]
        best_path.append(best_tag_id)
        
    # Pop off the start tag (we dont want to return that to the caller)
    start = best_path.pop()
    assert start == self.tag_to_ix[START_TAG] # Sanity check
    best_path.reverse()
    return path_score, best_path

In [25]:
def forward_calc(self, sentence, chars, chars2_length, d):
    
    '''
    The function calls viterbi decode and generates the 
    most probable sequence of tags for the sentence
    '''
    
    # Get the emission scores from the BiLSTM
    feats = self._get_lstm_features(sentence, chars, chars2_length, d)
    # viterbi to get tag_seq
    
    # Find the best path, given the features.
    if self.use_crf:
        score, tag_seq = self.viterbi_decode(feats)
    else:
        score, tag_seq = torch.max(feats, 1)
        tag_seq = list(tag_seq.cpu().data)

    return score, tag_seq

**Cài đặt mô hình**

In [26]:
def get_lstm_features(self, sentence, chars2, chars2_length, d):

    chars_embeds = self.char_embeds(chars2).unsqueeze(1)

    ## Creating Character level representation using Convolutional Neural Netowrk
    ## followed by a Maxpooling Layer
    chars_cnn_out3 = self.char_cnn3(chars_embeds)
    chars_embeds = nn.functional.max_pool2d(chars_cnn_out3,
                    kernel_size=(chars_cnn_out3.size(2), 1))
    chars_embeds = chars_embeds.view(chars_cnn_out3.size(0), self.out_channels)

    ## Loading word embeddings
    embeds = self.word_embeds(sentence)

    ## We concatenate the word embeddings and the character level representation
    ## to create unified representation for each word
    embeds = torch.cat((embeds, chars_embeds), 1)

    embeds = embeds.unsqueeze(1)

    ## Dropout on the unified embeddings
    embeds = self.dropout(embeds)

    ## Word lstm
    ## Takes words as input and generates a output at each step
    lstm_out, _ = self.lstm(embeds)

    ## Reshaping the outputs from the lstm layer
    lstm_out = lstm_out.view(len(sentence), self.hidden_dim*2)

    # Dropout on the lstm output
    lstm_out = self.dropout(lstm_out)

    ## Linear layer converts the ouput vectors to tag space
    lstm_feats = self.hidden2tag(lstm_out)
    
    return lstm_feats

In [27]:
def get_neg_log_likelihood(self, sentence, tags, chars2, chars2_length, d):
    # sentence, tags is a list of ints
    # features is a 2D tensor, len(sentence) * self.tagset_size
    feats = self._get_lstm_features(sentence, chars2, chars2_length, d)

    if self.use_crf:
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score
    else:
        tags = Variable(tags)
        scores = nn.functional.cross_entropy(feats, tags)
        return scores

In [28]:
class Print(nn.Module):
    def forward(self, x):
        print(x.size())
        return x

class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim,
                 char_to_ix=None, pre_word_embeds=None, char_out_dimension=25,char_embedding_dim=50, use_gpu=False
                 , use_crf=True, char_mode='CNN'):
        '''
        Input parameters:
                
                vocab_size= Size of vocabulary (int)
                tag_to_ix = Dictionary that maps NER tags to indices
                embedding_dim = Dimension of word embeddings (int)
                hidden_dim = The hidden dimension of the LSTM layer (int)
                char_to_ix = Dictionary that maps characters to indices
                pre_word_embeds = Numpy array which provides mapping from word embeddings to word indices
                char_out_dimension = Output dimension from the CNN encoder for character
                char_embedding_dim = Dimension of the character embeddings
                use_gpu = defines availability of GPU, 
                    when True: CUDA function calls are made
                    else: Normal CPU function calls are made
                use_crf = parameter which decides if you want to use the CRF layer for output decoding
        '''
        
        super(BiLSTM_CRF, self).__init__()
        
        #parameter initialization for the model
        self.use_gpu = use_gpu
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.use_crf = use_crf
        self.tagset_size = len(tag_to_ix)
        self.out_channels = char_out_dimension
        self.char_mode = char_mode

        if char_embedding_dim is not None:
            self.char_embedding_dim = char_embedding_dim
            
            #Initializing the character embedding layer
            self.char_embeds = nn.Embedding(len(char_to_ix), char_embedding_dim)
            init_embedding(self.char_embeds.weight)
                
            #Performing CNN encoding on the character embeddings
                
            layers = []

            layers.append(nn.Conv2d(in_channels=1, out_channels=25, kernel_size=(3, char_embedding_dim), padding=(2,0)))

            layers.append(nn.Conv2d(in_channels=self.out_channels, out_channels=self.out_channels, kernel_size=(3, 1), padding_mode='replicate'))

            net = nn.Sequential(*layers)

            self.char_cnn3 = net

        #Creating Embedding layer with dimension of ( number of words * dimension of each word)
        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        if pre_word_embeds is not None:
            #Initializes the word embeddings with pretrained word embeddings
            self.pre_word_embeds = True
            self.word_embeds.weight = nn.Parameter(torch.FloatTensor(pre_word_embeds))
        else:
            self.pre_word_embeds = False
    
        #Initializing the dropout layer, with dropout specificed in parameters
        self.dropout = nn.Dropout(parameters['dropout'])
        
        #Lstm Layer:
        #input dimension: word embedding dimension + character level representation
        #bidirectional=True, specifies that we are using the bidirectional LSTM
        self.lstm = nn.LSTM(embedding_dim+self.out_channels, hidden_dim, 2, bidirectional=True)
        
        
        #Initializing the lstm layer using predefined function for initialization
        init_lstm(self.lstm)
        
        # Linear layer which maps the output of the bidirectional LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim*2, self.tagset_size)
        
        #Initializing the linear layer using predefined function for initialization
        init_linear(self.hidden2tag) 

        if self.use_crf:
            # Matrix of transition parameters.  Entry i,j is the score of transitioning *to* i *from* j.
            # Matrix has a dimension of (total number of tags * total number of tags)
            self.transitions = nn.Parameter(
                torch.zeros(self.tagset_size, self.tagset_size))
            
            # These two statements enforce the constraint that we never transfer
            # to the start tag and we never transfer from the stop tag
            self.transitions.data[tag_to_ix[START_TAG], :] = -10000
            self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

    #assigning the functions, which we have defined earlier
    _score_sentence = score_sentences
    _get_lstm_features = get_lstm_features
    _forward_alg = forward_alg
    viterbi_decode = viterbi_algo
    neg_log_likelihood = get_neg_log_likelihood
    forward = forward_calc

In [29]:
#creating the model using the Class defined above
model = BiLSTM_CRF(vocab_size=len(word_to_id),
                   tag_to_ix=tag_to_id,
                   embedding_dim=parameters['word_dim'],
                   hidden_dim=parameters['word_lstm_dim'],
                   use_gpu=use_gpu,
                   char_to_ix=char_to_id,
                   pre_word_embeds=word_embeds,
                   use_crf=parameters['crf'],
                   char_mode=parameters['char_mode'])

/tmp/ipykernel_42/3399668671.py:6: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform(input_embedding, -bias, bias)
/tmp/ipykernel_42/2828957834.py:25: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform(weight, -sampling_range, sampling_range)
/tmp/ipykernel_42/2828957834.py:30: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform(weight, -sampling_range, sampling_range)
/tmp/ipykernel_42/2828957834.py:38: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform(weight, -sampling_range, sampling_range)
/tmp/ipykernel_42/2828957834.py:41: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform(weight, -sampling_range, sampling_range)


In [30]:
model.cuda()

BiLSTM_CRF(
  (char_embeds): Embedding(180, 50)
  (char_cnn3): Sequential(
    (0): Conv2d(1, 25, kernel_size=(3, 50), stride=(1, 1), padding=(2, 0))
    (1): Conv2d(25, 25, kernel_size=(3, 1), stride=(1, 1), padding_mode=replicate)
  )
  (word_embeds): Embedding(5728, 100)
  (dropout): Dropout(p=0.25, inplace=False)
  (lstm): LSTM(125, 200, num_layers=2, bidirectional=True)
  (hidden2tag): Linear(in_features=400, out_features=23, bias=True)
)

In [31]:
learning_rate = 0.001
number_of_epochs = parameters['epoch'] 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training

In [32]:
def train_loop(model):
    tr = time.time()
    model.train(True)
    eval_every = len(train_data)
    plot_every = 2000
    count = 0
    min_val_loss = 1000
    for epoch in range(1,number_of_epochs):
        total_loss_train = 0.0
        for i, index in enumerate(np.random.permutation(len(train_data))):
            data = train_data[index]

            ##gradient updates for each data entry
            model.zero_grad()

            sentence_in = data['words']
            sentence_in = Variable(torch.LongTensor(sentence_in))
            tags = data['tags']
            chars2 = data['chars']

            d = {}

            ## Padding the each word to max word size of that sentence
            chars2_length = [len(c) for c in chars2]
            char_maxl = max(chars2_length)
            chars2_mask = np.zeros((len(chars2_length), char_maxl), dtype='int')
            for i, c in enumerate(chars2):
                chars2_mask[i, :chars2_length[i]] = c
            chars2_mask = Variable(torch.LongTensor(chars2_mask))


            targets = torch.LongTensor(tags)

            #we calculate the negative log-likelihood for the predicted tags using the predefined function
            if use_gpu:
                neg_log_likelihood = model.neg_log_likelihood(sentence_in.cuda(), targets.cuda(), chars2_mask.cuda(), chars2_length, d)
            else:
                neg_log_likelihood = model.neg_log_likelihood(sentence_in, targets, chars2_mask, chars2_length, d)
            total_loss_train += neg_log_likelihood.data
            neg_log_likelihood.backward()

    #         #we use gradient clipping to avoid exploding gradients
    #         torch.nn.utils.clip_grad_norm(model.parameters(), gradient_clip)
            optimizer.step()

        total_loss_val=0.0
        model.train(False)
        for i, index in enumerate(np.random.permutation(len(dev_data))):
            data_val = dev_data[index]

            sentence_in = data_val['words']
            sentence_in = Variable(torch.LongTensor(sentence_in))
            tags = data_val['tags']
            chars2 = data_val['chars']

            d = {}

            ## Padding the each word to max word size of that sentence
            chars2_length = [len(c) for c in chars2]
            char_maxl = max(chars2_length)
            chars2_mask = np.zeros((len(chars2_length), char_maxl), dtype='int')
            for i, c in enumerate(chars2):
                chars2_mask[i, :chars2_length[i]] = c
            chars2_mask = Variable(torch.LongTensor(chars2_mask))


            targets = torch.LongTensor(tags)

            #we calculate the negative log-likelihood for the predicted tags using the predefined function
            if use_gpu:
                neg_log_likelihood = model.neg_log_likelihood(sentence_in.cuda(), targets.cuda(), chars2_mask.cuda(), chars2_length, d)
            else:
                neg_log_likelihood = model.neg_log_likelihood(sentence_in, targets, chars2_mask, chars2_length, d)
            total_loss_val += neg_log_likelihood.data

        print(
            f'Epochs: {epoch + 1} | Loss: {total_loss_train/len(train_data): .3f} | Val_Loss: {total_loss_val / len(dev_data): .3f} ')
        if total_loss_val / len(dev_data) < min_val_loss:
            min_val_loss = total_loss_val / len(dev_data)
            count = epoch
            print("Saving Model to ", model_name)
            torch.save(model.state_dict(), model_name)
        if epoch - count >= 5:
            return

        model.train(True)


    print(time.time() - tr)

train_loop(model)

#load pre-trained model
model.load_state_dict(torch.load(model_name))

Epochs: 2 | Loss:  7.023 | Val_Loss:  2.894 
Saving Model to  ./models/BCC_syllable
Epochs: 3 | Loss:  1.556 | Val_Loss:  2.187 
Saving Model to  ./models/BCC_syllable
Epochs: 4 | Loss:  0.968 | Val_Loss:  2.216 
Epochs: 5 | Loss:  0.667 | Val_Loss:  1.815 
Saving Model to  ./models/BCC_syllable
Epochs: 6 | Loss:  0.518 | Val_Loss:  1.800 
Saving Model to  ./models/BCC_syllable
Epochs: 7 | Loss:  0.417 | Val_Loss:  1.970 
Epochs: 8 | Loss:  0.322 | Val_Loss:  2.129 
Epochs: 9 | Loss:  0.263 | Val_Loss:  2.405 
Epochs: 10 | Loss:  0.232 | Val_Loss:  2.354 
Epochs: 11 | Loss:  0.182 | Val_Loss:  2.769 


<All keys matched successfully>

# Evaluate

In [33]:
def evaluate(model, test_data, id_to_tag):

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    predictions = []
    labels = []
    
    for data in test_data:

        words = data['str_words']
        chars2 = data['chars']

        d = {} 

        # Padding the each word to max word size of that sentence
        chars2_length = [len(c) for c in chars2]
        char_maxl = max(chars2_length)
        chars2_mask = np.zeros((len(chars2_length), char_maxl), dtype='int')
        for i, c in enumerate(chars2):
            chars2_mask[i, :chars2_length[i]] = c
        chars2_mask = Variable(torch.LongTensor(chars2_mask))

        dwords = Variable(torch.LongTensor(data['words']))

        # We are getting the predicted output from our model
        if use_gpu:
            _,predicted = model(dwords.cuda(), chars2_mask.cuda(), chars2_length, d)
            predictions.append([id_to_tag[val] for val in predicted])
            labels.append([id_to_tag[val] for val in data['tags']])
    print(classification_report(y_pred=predictions, y_true=labels, digits=3))

evaluate(model, test_data, id_to_tag)

                     precision    recall  f1-score   support

                AGE      0.967     0.967     0.967       582
               DATE      0.980     0.985     0.983      1654
             GENDER      0.955     0.961     0.958       462
                JOB      0.734     0.543     0.625       173
           LOCATION      0.932     0.921     0.927      4441
               NAME      0.944     0.843     0.890       318
       ORGANIZATION      0.840     0.838     0.839       771
         PATIENT_ID      0.982     0.971     0.976      2005
SYMPTOM_AND_DISEASE      0.897     0.739     0.811      1136
     TRANSPORTATION      0.896     0.933     0.914       193

          micro avg      0.939     0.912     0.925     11735
          macro avg      0.913     0.870     0.889     11735
       weighted avg      0.937     0.912     0.924     11735



# Inference

In [34]:
def ner(model, sentence, id_to_tag):


    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    #preprocessing
    s=sentence.split()
    str_words = [w for w in s]
    words = [word_to_id[w if w in word_to_id else '<UNK>'] for w in str_words]

    # Skip characters that are not in the training set
    chars = [[char_to_id[c] for c in w if c in char_to_id] for w in str_words]

    final_test={
        'str_words': str_words,
        'words': words,
        'chars': chars,
    }

    #prediction
    predictions = []
    words = final_test['str_words']
    chars2 = final_test['chars']

    d = {} 

    # Padding the each word to max word size of that sentence
    chars2_length = [len(c) for c in chars2]
    char_maxl = max(chars2_length)
    chars2_mask = np.zeros((len(chars2_length), char_maxl), dtype='int')
    for i, c in enumerate(chars2):
        chars2_mask[i, :chars2_length[i]] = c
    chars2_mask = Variable(torch.LongTensor(chars2_mask))

    dwords = Variable(torch.LongTensor(final_test['words']))

    # We are getting the predicted output from our model
    _,predicted_id = model(dwords.cuda(), chars2_mask.cuda(), chars2_length, d)

    prediction_label = [id_to_tag[val] for val in predicted_id]

    print(sentence)
    print(prediction_label)

In [35]:
ner(model,
    'Bệnh nhân nhập viện tối qua ở Bệnh Viện 115 là bệnh nhân thứ 82 , di chuyển qua nhiều thành phố bằng xe biển hiệu E-402',
    id_to_tag)

Bệnh nhân nhập viện tối qua ở Bệnh Viện 115 là bệnh nhân thứ 82 , di chuyển qua nhiều thành phố bằng xe biển hiệu E-402
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOCATION', 'I-LOCATION', 'I-LOCATION', 'O', 'O', 'O', 'O', 'B-PATIENT_ID', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-TRANSPORTATION']
